In [2]:

# install.packages('dplyr')
# install.packages('jsonlite')
# install.packages('lubridate')
# install.packages('ggplot2')
# install.packages('quantmod')

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



In [13]:
library(dplyr)
library(jsonlite)
library(lubridate)
library(ggplot2)
library(quantmod)
library(lifecycle)
library(tidyselect)



Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Loading required package: xts

Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



######################### Warning from 'xts' package ##########################
#                                                                             #
# The dplyr lag() function breaks how base R's lag() function is supposed to  #
# work, which breaks lag(my_xts). Calls to lag(my_xts) that you type or       #
# source() into this session won't work correctly.                            #
#                                                                             #
# Use stats::lag() to make sure you're not using dplyr::lag(), or you can add #
# conflictRules('dplyr', exclude = 'lag') to your .Rprofile to stop           #
# dplyr from breaking base R's lag() function.          

In [14]:
# Read the CSV file into a dataframe
race_df <- read.csv("csv_files/win_data_pro.csv")


# Remove rows where any entry in the trades column contains 0
race_df <- race_df %>% filter(trades != "[]")


race_df <- race_df %>%
  mutate(timestamp = as.POSIXct(timestamp_unix / 1000, origin = "1970-01-01", tz = "GMT"))


# Convert the string representations to actual lists
race_df$trades <- lapply(race_df$trades, function(x) fromJSON(as.character(x)))


# Function to calculate the size and price based on the given criteria
calculate_size_and_price <- function(lst) {
  n <- length(lst)

  if (n < 2) {
    return(list(size = NA, price = NA))
  }

  half_n <- ceiling(n / 2)

  # Elements in the first half
  first_half <- lst[1:half_n]

  # Elements in the last half
  last_half <- lst[(half_n + 1):n]

  # Sum the elements in the last half
  size <- sum(last_half)

  # Weighted average of the first half with weights being the corresponding elements in the last half
  if (length(first_half) == length(last_half)) {
    weighted_avg <- sum(first_half * last_half) / size
  } else {
    weighted_avg <- NA
  }

  return(list(size = size, price = weighted_avg))
}

# Apply the function to each element in the trades column and create new columns
results <- do.call(rbind, lapply(race_df$trades, calculate_size_and_price))


In [15]:
results


size    price   
 [1,] 9.99    3       
 [2,] 9.99    3       
 [3,] 10      3       
 [4,] 9.99    3       
 [5,] 13.4    3.25    
 [6,] 420.61  3.40047 
 [7,] 21.4    3.24243 
 [8,] 11.07   3.009756
 [9,] 420.61  3.40047 
[10,] 30      3.8     
[11,] 26.32   3.25    
[12,] 36.32   3.25    
[13,] 436.66  3.4     
[14,] 452.76  3.4     
[15,] 180.66  3.35    
[16,] 5.09    3.3     
[17,] 17.09   3.3     
[18,] 46.31   3.19607 
[19,] 654.46  3.383888
[20,] 80      3.3     
[21,] 5.99    3.85    
[22,] 10      3.25    
[23,] 177.82  3.3     
[24,] 117.09  3.3     
[25,] 136.32  3.25    
[26,] 156.32  3.25    
[27,] 4       34      
[28,] 4       3.9     
[29,] 180     3.3     
[30,] 4       19.1625 
[31,] ⋮       ⋮       
[32,] 2.95    120     
[33,] 6534.1  1.01    
[34,] 23.82   263.0898
[35,] 6536.98 1.01    
[36,] 28.91   117.2639
[37,] 6570.58 1.01    
[38,] 31.32   1000    
[39,] 6592.58 1.01    
[40,] 0.15    260     
[41,] 6598.94 1.01    
[42,] 6602.94 1.01    
[43,] 78.34   133.9431
[44,] 6654.44 1.01    
[45,] 6671.14 1.01    
[46,] 6691.14 1.01    
[47,] 7.01    945.4779
[48,] 8.78    1000    
[49,] 6696.22 1.01    
[50,] 6710.22 1.01    
[51,] 14.78   1000    
[52,] 6716.22 1.01    
[53,] 6721.98 1.01    
[54,] 6844.57 1.01    
[55,] 15.48   1000    
[56,] 0       NaN     
[57,] 0       NaN     
[58,] 0       NaN     
[59,] 0       NaN     
[60,] 0       NaN     
[61,] 0       NaN